In [2]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rcParams.update({'font.size':12})
from SimPEG import maps
import SimPEG.electromagnetics.time_domain as tdem
from SimPEG import utils
from ipywidgets import interactive, widgets

class EMSIMULATIONAPP(object):
    """EMSIMULATIONAPP"""
    def __init__(self):
        super(EMSIMULATIONAPP, self).__init__()
        self.n_layer = 3
        self._initialize()
        
    def _initialize(self):         
        stepoff_waveform = tdem.sources.StepOffWaveform()
        receiver_location = np.array([13.25, 0., 30.])
        receiver_orientation = "z"                    # "x", "y" or "z"
        times = np.logspace(-5, -2, 31)               # time channels

        receiver_list = [
            tdem.receivers.PointMagneticFluxTimeDerivative(
                receiver_location, times, orientation=receiver_orientation
            )
        ]

        source_location = np.array([0., 0., 30.])  
        source_radius = 10.
        current_amplitude = 1.
        source_orientation = 'z'
        source_list = []
        source_list.append(
            tdem.sources.MagDipole(
                receiver_list=receiver_list, location=source_location,
                waveform=stepoff_waveform, orientation=source_orientation
            )
        )
        survey = tdem.Survey(source_list)
        wire_map = maps.Wires(("sigma", self.n_layer), ("thickness", self.n_layer-1))
        self._simulation = tdem.Simulation1DLayered(
            survey=survey, thicknessesMap=wire_map.thickness, sigmaMap=wire_map.sigma,
        )
    @property
    def times(self):
        src = self._simulation.survey.source_list[0]
        return src.receiver_list[0].times
    
    def simulate(self, z, h, rho_background, rho_layer):
        thicknesses = np.array([z, h])        
        rho = np.array([rho_background, rho_layer, rho_background])
        sigma = 1./rho
        model = np.r_[sigma, thicknesses]
        dpred = self._simulation.dpred(model)
        return dpred
    
    def plot_data(self, z, h, rho_background, rho_layer, show):
        thickness = np.r_[z, h]
        rho = np.r_[rho_background, rho_layer, rho_background]
        rho0 = np.r_[rho_background, rho_background, rho_background]
        dpred = self.simulate(z, h, rho_background, rho_layer)
        dpred0 = self.simulate(z, h, rho_background, rho_background)
        difference = (dpred-dpred0) / dpred0 * 100
        detectability = np.sqrt((difference**2).sum()/len(difference))
        
        fig, axs = plt.subplots(1,2, figsize=(10, 5))
        utils.plot_1d_layer_model(thickness, rho0, ax=axs[0], **{'color':'C0', 'linestyle':'--'})
        utils.plot_1d_layer_model(thickness, rho, ax=axs[0], **{'color':'C0'})                
        if show=='data':
            axs[1].loglog(self.times * 1e3, -dpred, color='C0')
            axs[1].loglog(self.times * 1e3, -dpred0, color='C0', linestyle='--')
            axs[1].set_ylabel("Voltage (V/A-m$^4$)")
        elif show=='difference':
            axs[1].semilogx(self.times * 1e3, difference, color='C0')
            axs[1].set_ylabel("Relative difference (%)")
        axs[1].set_title("Detectability={:.1f}%".format(detectability))
        axs[1].set_xlabel("Time (ms)")    
        axs[0].set_xlabel("Resistivity (Ohm-m)")
        axs[1].grid(which='both', alpha=0.2)
        axs[0].set_title("Vertical resistivity profile")
        plt.tight_layout()
        plt.show()
        return     
    
    def interact_aem(self):
        Q = interactive(
            self.plot_data, 
            z=widgets.FloatText(value=20, description='$z$'),
            h=widgets.FloatText(value=20, description='$h$'),
            rho_background=widgets.FloatText(value=20, description='$\\rho_b$'),
            rho_layer=widgets.FloatText(value=50, description='$\\rho_{\\text{layer}}$'),
            show=widgets.RadioButtons(options=["data", "difference"])
        )
        return Q

In [3]:
app  = EMSIMULATIONAPP()

## Airborne EM simulation app

With this web application you can simulate signals from an airborne EM experiment.
A layer is embedded in a homogenous background. Parameters defining the subsurface resistivity are:

- $z$: depth the the top of the layer 
- $h$: thickness of the layer
- $\rho_{b}$: resistivity of the background
- $\rho_{layer}$: resistivity of the layer

In [4]:
app.interact_aem()

interactive(children=(FloatText(value=20.0, description='$z$'), FloatText(value=20.0, description='$h$'), Floa…